#HW3 Classification

1. Implement VGG16

https://neurohive.io/en/popular-networks/vgg16/

2. Implement Data Augmentation

3. Implement Test Time Augmentation


#Result

![替代文字](https://drive.google.com/uc?id=17RHUJszLdDXSlqA6nUHsY_PL28ClHzqg)

Rank 20/293

![替代文字](https://drive.google.com/uc?id=1R1FhTcZLDJQhBQR-dOdbWMuYRMKfs_v9)

下載資料集

In [ ]:
!gdown --id '19CzXudqN58R3D-1G8KeFWk8UDQwlb8is' --output food-11.zip # 下載資料集
!unzip food-11.zip # 解壓縮

串流輸出內容已截斷至最後 5000 行。
  inflating: food-11/training/4_165.jpg  
  inflating: food-11/training/5_376.jpg  
  inflating: food-11/training/2_691.jpg  
  inflating: food-11/training/0_541.jpg  
  inflating: food-11/training/3_482.jpg  
  inflating: food-11/training/0_227.jpg  
  inflating: food-11/training/5_410.jpg  
  inflating: food-11/training/4_603.jpg  
  inflating: food-11/training/8_341.jpg  
  inflating: food-11/training/5_1154.jpg  
  inflating: food-11/training/9_37.jpg  
  inflating: food-11/training/9_152.jpg  
  inflating: food-11/training/5_438.jpg  
  inflating: food-11/training/9_1287.jpg  
  inflating: food-11/training/8_369.jpg  
  inflating: food-11/training/2_1455.jpg  
  inflating: food-11/training/10_247.jpg  
  inflating: food-11/training/7_32.jpg  
  inflating: food-11/training/10_521.jpg  
  inflating: food-11/training/2_1333.jpg  
  inflating: food-11/training/2_861.jpg  
  inflating: food-11/training/0_569.jpg  
  inflating: food-11/training/6_289.jpg  
  inflati

In [ ]:
!pip freeze | grep albumentations
!pip install albumentations

albumentations==0.1.12
     |████████████████████████████████| 634kB 4.5MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=58f769c4fc18eee8a74bed6d05b588ae8d54aa6c52d1ed22c011c6284baaf599
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [ ]:
# Import需要的套件
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import time

#Read image
利用 OpenCV (cv2) 讀入照片並存放在 numpy array 中

In [ ]:
def readfile(path, label):
    # label 是一個 boolean variable，代表需不需要回傳 y 值
    image_dir = sorted(os.listdir(path))
    x = np.zeros((len(image_dir), 224, 224, 3), dtype=np.uint8)
    y = np.zeros((len(image_dir)), dtype=np.uint8)
    for i, file in enumerate(image_dir):
        img = cv2.imread(os.path.join(path, file))
        x[i, :, :] = cv2.resize(img,(224, 224))
        if label:
          y[i] = int(file.split("_")[0])
    if label:
      return x, y
    else:
      return x

In [ ]:
# 分別將 training set、validation set、testing set 用 readfile 函式讀進來
workspace_dir = './food-11'
print("Reading data")
train_x, train_y = readfile(os.path.join(workspace_dir, "training"), True)
print("Size of training data = {}".format(len(train_x)))
val_x, val_y = readfile(os.path.join(workspace_dir, "validation"), True)
print("Size of validation data = {}".format(len(val_x)))
test_x = readfile(os.path.join(workspace_dir, "testing"), False)
print("Size of Testing data = {}".format(len(test_x)))

Reading data
Size of training data = 9866
Size of validation data = 3430
Size of Testing data = 3347


#Dataset Augmentation
用albumentations 做資料強化

In [ ]:
from albumentations import Compose, Normalize, ShiftScaleRotate, HorizontalFlip, PadIfNeeded, CenterCrop, VerticalFlip, Rotate

class AugmentedDataset(Dataset):
    def __init__(self, feature, label=None, train_mean=None, train_std=None, transform=[]):
        self.feature = feature
        self.label = label
        self.train_mean = train_mean
        self.train_std = train_std
        self.transform = transform
        if label is not None:
            self.label = torch.LongTensor(label)
    
    def __getitem__(self, idx):
        feature = self.feature[idx]
        if self.train_mean is not None and self.train_std is not None:
            feature = (feature - self.train_mean) / self.train_std
        feature = Compose(self.transform)(image=feature)['image']
        feature = np.transpose(feature, (2, 0, 1))
        if self.label is not None:
            return feature, self.label[idx]
        else:
            return feature
    
    def __len__(self):
        return self.feature.shape[0]

class TTADataset(Dataset):
    def __init__(self, feature, label=None, train_mean=None, train_std=None, transform=[]):
        self.feature = feature
        self.label = label
        self.train_mean = train_mean
        self.train_std = train_std
        self.transform = transform
        if label is not None:
            self.label = torch.LongTensor(label)
    
    def __getitem__(self, idx):
        feature = self.feature[idx]
        if self.train_mean is not None and self.train_std is not None:
            feature = (feature - self.train_mean) / self.train_std
        features = [feature]
        features.append(HorizontalFlip(always_apply=True)(image=feature)['image'])
        features.append(VerticalFlip(always_apply=True)(image=feature)['image'])
        features = np.stack(features, axis=0)
        features = np.transpose(features, (0, 3, 1, 2))
        if self.label is not None:
            return features, self.label[idx]
        else:
            return features
    
    def __len__(self):
        return self.feature.shape[0]

In [ ]:
#normalization
train_mean = np.mean(train_x, axis=(0, 1, 2))
train_std = 0.0
for i in range(train_x.shape[0]):
    train_std += np.std(train_x[i], axis=(0, 1))
train_std /= train_x.shape[0]
print(train_mean, train_std)


[ 87.48389569 114.84202727 141.35665198] [61.14641939 61.35785275 58.70858843]


In [ ]:
transform = [
    ShiftScaleRotate(shift_limit=0.0, scale_limit=0.2, rotate_limit=45, p=0.5),
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.5),
    # GaussianBlur(blur_limit=3, p=1.0),  #  如果圖片大一點可以考慮加 Gaussian Blur
    PadIfNeeded(min_height=128, min_width=128, p=1.0),
    CenterCrop(height=224, width=224),
]

batch_size = 32
train_set = AugmentedDataset(train_x, train_y, train_mean=train_mean, train_std=train_std, transform=transform)
val_set = TTADataset(val_x, val_y, train_mean=train_mean, train_std=train_std)
test_set = TTADataset(test_x, train_mean=train_mean, train_std=train_std)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

#Model Defination

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 224, 224]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 224, 224]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, 1, 1),  # [64, 224, 224]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 112, 112]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 112, 112]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, 1, 1), # [128, 112, 112]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 56, 56]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 56, 56]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), # [256, 56, 56]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), # [256, 56, 56]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 28, 28]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 28, 28]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 28, 28]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 28, 28]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 14, 14]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 14, 14]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 14, 14]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 14, 14]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 7, 7]
        )
        self.fc = nn.Sequential(
            nn.Linear(512*7*7, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(256, 11)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

In [ ]:
class Classifier2(nn.Module):
    def __init__(self):
        super(Classifier2, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        self.fc = nn.Sequential(
            nn.Linear(512*7*7, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 11)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

#Training

Baseline

Train Acc: 0.871782 Loss: 0.002866 | Val Acc: 0.628863 loss: 0.012328

After normalization and TTA

Train Acc: 0.874316 Loss: 0.002828 | Val Acc: 0.695044 loss: 0.008962

Use VGG16 Artichture, change image input size to 224x224x3, bach size=32, lr = 0.0001

[113/120] 145.41 sec(s) Train Acc: 0.952767 Loss: 0.004681 | Val Acc: 0.838484 loss: 0.023540

Use More DTA and TTA


In [ ]:
model = Classifier().cuda()
loss = nn.CrossEntropyLoss() # 因為是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
#exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)
num_epoch = 120
best_val_acc = 0
best_model = model

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    model.train() # 確保 model 是在 train model (開啟 Dropout 等...)
    for i, data in enumerate(train_loader):
        feature = data[0]
        label = data[1]
        feature = feature.to(torch.device('cuda')).float()
        label = label.to(torch.device('cuda'))
        #print(feature.shape)

        optimizer.zero_grad() # 用 optimizer 將 model 參數的 gradient 歸零
        train_pred = model(feature) # 利用 model 得到預測的機率分佈 這邊實際上就是去呼叫 model 的 forward 函數
        batch_loss = loss(train_pred, label) # 計算 loss （注意 prediction 跟 label 必須同時在 CPU 或是 GPU 上）
        batch_loss.backward() # 利用 back propagation 算出每個參數的 gradient
        optimizer.step() # 以 optimizer 用 gradient 更新參數值

        train_pred = np.argmax(train_pred.detach().cpu().numpy(), axis=1)

        train_acc += np.sum(train_pred == label.cpu().numpy())
        train_loss += batch_loss.item()
    #exp_lr_scheduler.step()
    
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
          feature = data[0].to(torch.device('cuda')).float()
          label = data[1].to(torch.device('cuda'))
          with torch.set_grad_enabled(False):
              if len(feature.size()) == 5:  # test time augmentation 用
                  B, T, C, H, W = feature.size()
                  output = model(feature.view(-1, C, H, W))
                  output = output.view(B, T, -1)
                  output = output.mean(dim=1)
              else:
                  output = model(feature)
          val_pred = output
          batch_loss = loss(val_pred, label)

          val_pred = np.argmax(val_pred.detach().cpu().numpy(), axis=1)
          val_acc += np.sum(val_pred == label.cpu().numpy())
          val_loss += batch_loss.item()

        #將結果 print 出來
        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
            (epoch + 1, num_epoch, time.time()-epoch_start_time, \
             train_acc/train_set.__len__(), train_loss/train_set.__len__(), val_acc/val_set.__len__(), val_loss/val_set.__len__()))
    
    if val_acc > best_val_acc:
      best_val_acc = val_acc
      best_model = model
      
torch.save(best_model.state_dict(), 'content/drive/My Drive/hw3_best.pt')


[001/120] 146.26 sec(s) Train Acc: 0.223799 Loss: 0.069371 | Val Acc: 0.286589 loss: 0.063644
[002/120] 146.21 sec(s) Train Acc: 0.286033 Loss: 0.064322 | Val Acc: 0.312828 loss: 0.062293
[003/120] 145.98 sec(s) Train Acc: 0.310866 Loss: 0.061784 | Val Acc: 0.341983 loss: 0.058689
[004/120] 146.01 sec(s) Train Acc: 0.354551 Loss: 0.058250 | Val Acc: 0.405831 loss: 0.054568
[005/120] 146.04 sec(s) Train Acc: 0.378167 Loss: 0.056414 | Val Acc: 0.445190 loss: 0.051574
[006/120] 145.98 sec(s) Train Acc: 0.407460 Loss: 0.054470 | Val Acc: 0.457434 loss: 0.049437
[007/120] 145.88 sec(s) Train Acc: 0.427732 Loss: 0.052283 | Val Acc: 0.499125 loss: 0.046856
[008/120] 145.82 sec(s) Train Acc: 0.454490 Loss: 0.050321 | Val Acc: 0.517493 loss: 0.046414
[009/120] 145.77 sec(s) Train Acc: 0.484898 Loss: 0.048124 | Val Acc: 0.474344 loss: 0.049378
[010/120] 145.55 sec(s) Train Acc: 0.503446 Loss: 0.046546 | Val Acc: 0.549854 loss: 0.042318
[011/120] 145.74 sec(s) Train Acc: 0.518143 Loss: 0.044750 |

In [ ]:
train_val_x = np.concatenate((train_x, val_x), axis=0)
train_val_y = np.concatenate((train_y, val_y), axis=0)

train_val_set = AugmentedDataset(train_val_x, train_val_y, train_mean=train_mean, train_std=train_std, transform=transform)
train_val_loader = DataLoader(train_val_set, batch_size=batch_size, shuffle=True)

In [ ]:
model_best = Classifier().cuda()
loss = nn.CrossEntropyLoss() # 因為是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model_best.parameters(), lr=1e-4) # optimizer 使用 Adam
num_epoch = 120

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    model_best.train() # 確保 model 是在 train model (開啟 Dropout 等...)
    for i, data in enumerate(train_val_loader):
        feature = data[0]
        label = data[1]
        feature = feature.to(torch.device('cuda')).float()
        label = label.to(torch.device('cuda'))
        #print(feature.shape)

        optimizer.zero_grad() # 用 optimizer 將 model 參數的 gradient 歸零
        train_pred = model_best(feature) # 利用 model 得到預測的機率分佈 這邊實際上就是去呼叫 model 的 forward 函數
        batch_loss = loss(train_pred, label) # 計算 loss （注意 prediction 跟 label 必須同時在 CPU 或是 GPU 上）
        batch_loss.backward() # 利用 back propagation 算出每個參數的 gradient
        optimizer.step() # 以 optimizer 用 gradient 更新參數值

        train_pred = np.argmax(train_pred.detach().cpu().numpy(), axis=1)

        train_acc += np.sum(train_pred == label.cpu().numpy())
        train_loss += batch_loss.item()

        #將結果 print 出來
    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f' % \
      (epoch + 1, num_epoch, time.time()-epoch_start_time, \
      train_acc/train_val_set.__len__(), train_loss/train_val_set.__len__()))

[001/120] 245.84 sec(s) Train Acc: 0.241501 Loss: 0.067867
[002/120] 250.25 sec(s) Train Acc: 0.302798 Loss: 0.063030
[003/120] 249.66 sec(s) Train Acc: 0.340328 Loss: 0.059857
[004/120] 249.32 sec(s) Train Acc: 0.369134 Loss: 0.057275
[005/120] 249.56 sec(s) Train Acc: 0.403730 Loss: 0.054811
[006/120] 249.61 sec(s) Train Acc: 0.431032 Loss: 0.052283
[007/120] 249.56 sec(s) Train Acc: 0.453069 Loss: 0.050631
[008/120] 249.59 sec(s) Train Acc: 0.482852 Loss: 0.048202
[009/120] 249.42 sec(s) Train Acc: 0.496315 Loss: 0.046732
[010/120] 249.43 sec(s) Train Acc: 0.520156 Loss: 0.044571
[011/120] 249.49 sec(s) Train Acc: 0.535199 Loss: 0.043565
[012/120] 249.09 sec(s) Train Acc: 0.546856 Loss: 0.041914
[013/120] 250.33 sec(s) Train Acc: 0.569344 Loss: 0.040800
[014/120] 250.63 sec(s) Train Acc: 0.581603 Loss: 0.039278
[015/120] 250.30 sec(s) Train Acc: 0.591306 Loss: 0.038058
[016/120] 250.05 sec(s) Train Acc: 0.613418 Loss: 0.036759
[017/120] 250.43 sec(s) Train Acc: 0.625677 Loss: 0.0356

In [ ]:
model_best.eval()
prediction = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        feature = data
        feature = feature.to(torch.device('cuda')).float()

        with torch.set_grad_enabled(False):
              if len(feature.size()) == 5:  # test time augmentation 用
                  B, T, C, H, W = feature.size()
                  output = model_best(feature.view(-1, C, H, W))
                  output = output.view(B, T, -1)
                  output = output.mean(dim=1)
              else:
                  output = model_best(feature)
        test_pred = output
        test_label = np.argmax(test_pred.detach().cpu().numpy(), axis=1)

        for y in test_label:
            prediction.append(y)

In [ ]:
#將結果寫入 csv 檔
with open("predict.csv", 'w') as f:
    f.write('Id,Category\n')
    for i, y in  enumerate(prediction):
        f.write('{},{}\n'.format(i, y))

Baseline

Private: 0.73237  Public: 0.72145

After normalization and TTA

Private: 0.73954  Public: 0.72803


In [ ]:
!gdown --id 1VoF-D1FH0PhIne0pdDMKNmSg-SXfBoKS
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions submit -c ml2020spring-hw3 -f predict.csv -m "Message"